In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import time

In [ ]:
# file paths
trn_path = './data/train_ver2.csv'
tst_path = './data/test_ver2.csv'
labels_path = './data/labels.csv'

print('# Loading data to generate lag..')
# load full data
trn = pd.read_csv(trn_path)
tst = pd.read_csv(tst_path)
labels = pd.read_csv(labels_path).astype(int)

# set lag dates
trn_dates = ['2015-01-28', '2015-02-28', '2015-03-28', '2015-04-28', '2015-05-28']
tst_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']

print('# Selecting ncodpers..')
# select ncodpers
temp = trn[trn['fecha_dato'] == '2015-06-28']['ncodpers']
trn_ncodpers = temp[(labels[trn['fecha_dato'] == '2015-06-28'].sum(axis=1) > 0).values].values.tolist()
tst_ncodpers = np.unique(tst['ncodpers']).tolist()

print('# Trimming data..')
# trim lag data with given date, given ncodpers
trn_trim = trn[trn['fecha_dato'].isin(trn_dates)]
trn_trim = trn_trim[trn_trim['ncodpers'].isin(trn_ncodpers)]
tst_trim = trn[trn['fecha_dato'].isin(tst_dates)]
tst_trim = tst_trim[tst_trim['ncodpers'].isin(tst_ncodpers)]

print('# Melting data into 24-class..')
# melt target into single 24-class
fecha_dato = trn['fecha_dato']
train_index = (labels[fecha_dato == '2015-06-28'].sum(axis=1) > 0)
train_index = train_index[train_index == True]
train = trn.ix[train_index.index]
train.iloc[:, 24:] = labels.ix[train_index.index]

trn_june = []
for ind, (run, row) in enumerate(train.iterrows()):
    for i in range(24):
        if row[24 + i] == 1:
            temp = row[:24].values.tolist()
            temp.append(i)
            trn_june.append(temp)

# define and save target separately
trg = pd.DataFrame(trn_june)[24].values.tolist()
trg = pd.DataFrame(trg)

# make full data set
trn_june = pd.DataFrame(trn_june, columns=trn.columns[:25]).iloc[:, :-1]
trn = pd.concat([trn_trim, trn_june], axis=0)
tst = pd.concat([tst_trim, tst], axis=0)
print('# intermediate : selected necessary rows only..')
print('# trn : {} | target : {} | tst : {}'.format(trn.shape, trg.shape, tst.shape))

print('# Saving trn_lag, tst_lag, target in csv..')

In [ ]:
# clean data
skip_cols = ['fecha_dato', 'ncodpers']
target_cols = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
               'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
               'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
               'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
               'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
               'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
               'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
               'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

for col in trn.columns:
    if col in skip_cols:
        continue
    print('# column : {}'.format(col))

    if col == 'ind_empleado':
        trn[col].fillna('S', inplace=True)
    elif col == 'age':
        trn[col].replace(' NA', 0, inplace=True)
        trn[col] = trn[col].astype(str).astype(int)
        trn[col] = trn[col].astype(str).astype(int)
        continue
    elif col == 'fecha_alta':
        trn[col] = ((pd.to_datetime(trn['fecha_dato']) - pd.to_datetime(trn[col].fillna('2015-07-01')))
                    / np.timedelta64(1,'D')).astype(int)
        tst[col] = ((pd.to_datetime(tst['fecha_dato']) - pd.to_datetime(tst[col]))
                    / np.timedelta64(1, 'D')).astype(int)
        continue
    elif col == 'antiguedad':
        trn[col].replace('     NA', -1, inplace=True)
        trn[col] = trn[col].astype(str).astype(int)
        tst[col] = tst[col].astype(str).astype(int)
        continue
    elif col == 'ult_fec_cli_1t':
        trn[col] = ((pd.to_datetime(trn['fecha_dato']) - pd.to_datetime(trn[col].fillna('2015-06-30')))
                    / np.timedelta64(1,'D')).astype(int)
        tst[col] = ((pd.to_datetime(tst['fecha_dato']) - pd.to_datetime(tst[col].fillna('2016-01-03')))
                    / np.timedelta64(1,'D')).astype(int)
        continue
    elif col == 'indrel_1mes':
        tst[col].replace('1', '1.0', inplace=True)
        tst[col].replace('2', '1.0', inplace=True)
        tst[col].replace('2.0', '1.0', inplace=True)
        tst[col].replace(2.0, '1.0', inplace=True)
        tst[col].replace('3', '3.0', inplace=True)
        tst[col].replace('4', '3.0', inplace=True)
        tst[col].replace(4.0, '3.0', inplace=True)
        tst[col].replace('4.0', '3.0', inplace=True)
        tst[col].replace('P', '3.0', inplace=True)
    elif col == 'tiprel_1mes':
        tst[col].replace('N', 'I', inplace=True)
        tst[col].replace('R', 'P', inplace=True)
    elif col == 'indresi':
        trn[col].fillna('N', inplace=True)
    elif col == 'indext':
        trn[col].fillna('S', inplace=True)
    elif col == 'indfall':
        trn[col].fillna('N', inplace=True)
    elif col == 'tipodom':
        trn.drop([col], axis=1, inplace=True)
        tst.drop([col], axis=1, inplace=True)
        continue
    elif col == 'ind_actividad_cliente':
        trn[col].fillna(0.0, inplace=True)
    elif col == 'renta':
        tst[col].replace('         NA', 0, inplace=True)
        trn[col].fillna(-1, inplace=True)
        tst[col].fillna(-1, inplace=True)
        trn[col] = trn[col].astype(str).astype(float).astype(int)
        tst[col] = tst[col].astype(str).astype(float).astype(int)
        continue
    elif col in target_cols:
        trn[col].fillna(0, inplace=True)
        trn[col] = trn[col].astype(int)
        tst[col].fillna(0, inplace=True)
        tst[col] = tst[col].astype(int)

    lb = LabelEncoder()
    lb.fit(pd.concat([trn[col].astype(str), tst[col].astype(str)], axis=0))
    trn[col] = lb.transform(trn[col].astype(str))
    tst[col] = lb.transform(tst[col].astype(str))

print('# Appending lag-5..')
trn_june = trn[trn['fecha_dato'] == '2015-06-28'].drop(target_cols, axis=1)
trn_othr = trn[trn['fecha_dato'] != '2015-06-28']
tst_june = tst[tst['fecha_dato'] == '2016-06-28'].drop(target_cols, axis=1)
tst_othr = tst[tst['fecha_dato'] != '2016-06-28']

drop_cols = ['fecha_dato', 'ncodpers']

print('# Appending trn data.. {} rows'.format(trn_june.shape[0]))
st = time.time()
trn_append = []
for i, ncodper in enumerate(trn_june['ncodpers']):
    temp = trn_othr[trn_othr['ncodpers'] == ncodper].drop(drop_cols, axis=1)
    if temp.shape[0] == 0:
        row = ['NA'] * 225
    else:
        row = np.hstack([temp.shift(periods=i).iloc[-1, :] for i in range(temp.shape[0])]).tolist()
    trn_append.append(trn_june.iloc[i].drop(drop_cols).values.tolist() + row)

    if i % int(trn_june.shape[0] / 10) == 0:
        print('# {} rows.. {} secs..'.format(i, round(time.time() - st), 2))

In [ ]:
print('# Appending tst data.. {} rows'.format(tst_june.shape[0]))
st = time.time()
tst_append = []
for i, ncodper in enumerate(tst_june['ncodpers']):
    temp = tst_othr[tst_othr['ncodpers'] == ncodper].drop(drop_cols, axis=1)
    if temp.shape[0] == 0:
        row = ['NA'] * 225
    else:
        row = np.hstack([temp.shift(periods=i).iloc[-1, :] for i in range(temp.shape[0])]).tolist()
    tst_append.append(tst_june.iloc[i].drop(drop_cols).values.tolist() + row)

    if i % int(tst_june.shape[0] / 10) == 0:
        print('# {} rows.. {} secs..'.format(i, round(time.time() - st), 2))

# change colnames
colnames = trn_june.drop(drop_cols, axis=1).columns.values.tolist()
suffixes = ['_lag_one', '_lag_two', '_lag_thr', '_lag_fou', '_lag_fiv']
for suffix in suffixes:
    for col in trn_othr.drop(drop_cols, axis=1).columns.values.tolist():
        colnames.append(col + suffix)

trn = pd.DataFrame(trn_append, columns=colnames)
tst = pd.DataFrame(tst_append, columns=colnames)
print('# trn : {} | tst : {}'.format(trn.shape, tst.shape))

In [ ]:
lags = ['_lag_one', '_lag_two', '_lag_thr', '_lag_fou', '_lag_fiv']
diffs = [['fiv', 'fou'], ['fou', 'thr'], ['thr', 'two'], ['two', 'one']]

print('# na_count')
# null count per row
trn['na_count'] = trn.isnull().sum(axis=1)
tst['na_count'] = tst.isnull().sum(axis=1)

print('# target_sum_lag')
# total count of purchases per month
for lag in lags:
    trn['target_sum' + lag] = (trn[[col + lag for col in target_cols]].sum(axis=1))
    tst['target_sum' + lag] = (tst[[col + lag for col in target_cols]].sum(axis=1))

print('# avg of cols')
# average of cols over past 5 months
cols = ['ind_actividad_cliente', 'ult_fec_cli_1t']
for col in cols:
    trn[col + lag + '_avg'] = (trn[[col + lag for lag in lags]]).mean(axis=1)
    tst[col + lag + '_avg'] = (tst[[col + lag for lag in lags]]).mean(axis=1)

print('# target_sum over lag-5')
# cumulative sum of target cols over past 5 months
for col in target_cols:
    trn[col + '_sum'] = (trn[[col + lag for lag in lags]].sum(axis=1))
    tst[col + '_sum'] = (tst[[col + lag for lag in lags]].sum(axis=1))

print('# target_sum_diff for each months')
# change in count of purchases per month compared to its last month
for diff in diffs:
    pre = diff[0]
    post = diff[1]
    trn['target_diff_' + post + '-' + pre] = trn['target_sum_lag_' + post] - trn['target_sum_lag_' + pre]
    tst['target_diff_' + post + '-' + pre] = tst['target_sum_lag_' + post] - tst['target_sum_lag_' + pre]

print('# target_diff for each months')
# change in individual purchases for each month compared to its last month
for col in target_cols:
    for diff in diffs:
        pre = diff[0]
        post = diff[1]
        trn[col + '_label_lag_' + post] = trn[col + '_lag_' + post] - trn[col + '_lag_' + pre]
        tst[col + '_label_lag_' + post] = tst[col + '_lag_' + post] - tst[col + '_lag_' + pre]

print('# unique target count')
# unique count of purchased targets over 5 months
trn['unique_target_count'] = (trn[[col + '_sum' for col in target_cols]] > 0).astype(int).sum(axis=1)
tst['unique_target_count'] = (tst[[col + '_sum' for col in target_cols]] > 0).astype(int).sum(axis=1)

print('# Drop infrequent targets..')
rem_targets = [2, 23, 22, 21, 18, 17, 4, 12, 11, 9, 6, 13, 7, 19, 8]
trn = trn[trg['0'].isin(rem_targets)]
trg = trg[trg['0'].isin(rem_targets)]
trg = LabelEncoder().fit_transform(trg)

print('# trn : {} | trg : {} | tst : {}'.format(trn.shape, trg.shape, tst.shape))

# cache
print('# Caching data as trn.csv / tst.csv ..')
trn.to_csv('data/adv.feature_engineer.trn.csv', index=False)
tst.to_csv('data/adv.feature_engineer.tst.csv', index=False)
pd.DataFrame(trg).to_csv('data/adv.feature_engineer.y.csv', index=False)